In [36]:
import pandas as pd
from neupy import algorithms
import numpy as np
import json
from tqdm import tqdm

In [37]:
def ret(n, n_1):
    return (n_1 - n) / n


# prices every 15 seconds
# df = pd.read_csv("prices.csv")
f = open("BTC_USDT-1m.json")
df = pd.DataFrame.from_records(json.load(f), columns=['timestamp', 'open', 'high', 'low', 'close', 'vol'])
# df.columns = ['price']
df['ret'] = (df['high'].shift(-1) - df['open']) / df['open']
df = df.iloc[:3600]

In [38]:
df

,timestamp,open,high,low,close,vol,ret
0,1643760000000,38694.59,38739.04,38692.56,38722.39,16.39465,0.002372
1,1643760060000,38724.92,38786.36,38714.47,38763.09,14.60847,0.001117
2,1643760120000,38763.08,38768.18,38715.64,38746.91,28.64245,-0.000221
3,1643760180000,38746.90,38754.50,38725.00,38728.05,15.48239,-0.000486
4,1643760240000,38728.05,38728.05,38714.00,38720.95,11.37676,0.000062
...,...,...,...,...,...,...,...
3595,1643975700000,37829.41,37838.43,37818.72,37825.20,7.43072,0.000412
3596,1643975760000,37825.21,37844.99,37818.72,37844.84,21.59401,0.000519
3597,1643975820000,37844.83,37844.84,37828.11,37831.90,8.03157,-0.000260
3598,1643975880000,37831.90,37835.00,37819.01,37825.44,16.43789,-0.000078


In [50]:
max_hit_rate = -1
std = 0.0037GF                       ----------------------
for std in tqdm(np.arange(0.0036, 0.0039, 0.000001)):
    predictions = []
    for i in range(len(df) - NDTR) :
        small_df = df.iloc[i: i + NDTR].copy(deep=True)[['open', 'high', 'ret']]
        small_df['open'] = small_df['open'] / small_df['open'].mean()
        small_df['high'] = small_df['high'] / small_df['high'].mean()
        x_train = small_df.iloc[0:NDTR - 1][['open', 'high']].values
        y_train = small_df['ret'].iloc[0:NDTR - 1]
        grnn = algorithms.GRNN(std=std, verbose=False)
        grnn.train(x_train, y_train.values)

        x_predict = small_df.iloc[NDTR - 1][['open', 'high']].values
        prediction = grnn.predict([x_predict])
        predictions.append(prediction[0][0])
    y_test= df['ret'].iloc[NDTR:].values
    hits = 0
    for i in range(len(predictions)):
        if np.sign(y_test[i]) == np.sign(predictions[i]):
            hits+=1
    hit_rate = hits / len(predictions)
    if hit_rate > max_hit_rate:
        max_hit_rate = hit_rate
        print("found max hit_rate", hit_rate, "std:", std, "spredictions=", len(predictions))

  0%|          | 1/300 [00:07<39:22,  7.90s/it]

found max hit_rate 0.7302725968436155 std: 0.0036 predictions= 3485


100%|██████████| 300/300 [39:08<00:00,  7.83s/it]
